In [129]:
import pandas as pd
import os
import time
import fnmatch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import random

datapath = '/home/jxin05/Desktop/EXP/smooth_50_10/'

In [130]:
# pick 900 pure samples and 1000 non-pure samples used for training
# 100 pure samples and 100 non-pure smaples used for validation
def BuildTrainValidset(Xpure, X_100, X_200, X_300, X_400, X_500):
    trainX = np.vstack([Xpure[:900, :, :], X_100[:200, :, : ], X_200[:200, :, : ], X_300[:200, :, : ], X_400[:200, :, : ],X_500[:200, :, : ]])
    trainYpure = np.zeros((900, 1))
    trainYsyn = np.ones((1000, 1))
    trainY = np.vstack([trainYpure, trainYsyn])
    
    validX = np.vstack([Xpure[900:, :, :], X_100[-20:, :, : ], X_200[-20:, :, : ], X_300[-20:, :, : ], X_400[-20:, :, : ],X_500[-20:, :, : ]])
    validYpure = np.zeros((100, 1))
    validYsyn = np.ones((100, 1))
    validY = np.vstack([validYpure, validYsyn])
    return trainX, trainY, validX, validY

def predict(net, x):
    with torch.no_grad():
        h = net.init_hidden(1)
        output, h = net(x,h)
    if(output[0,-1,0] >= output[0, -1, 1]):
        return 0
    else:
        return 1
    
def check_acc(net, X, Y):
    correct = 0
    wrong = 0
    for i in range(X.shape[0]):
        res = predict(net,X[i])
        if(Y[i] == res):
            correct += 1
        else:
            wrong += 1
    return correct/X.shape[0]

def take_batch(batch_size, X, Y):
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    for i in range(0,X.shape[0]-batch_size+1, batch_size):
        excerpt = indices[i:i + batch_size]
        yield X[excerpt], Y[excerpt]

# reshape data to fit network
# input size is (n x 1000 x 3)
# output size is (n x 12 x 1 x 3 x 100) - 12 timesteps
def transformation(X, overlap = 0.2, stepsize = 100):
    if(len(X.shape) != 3):
        X = np.array([X])
    start = 0
    length = X.shape[1]
    num_step = 0
    x_lst = []
    while(start + stepsize <= length - 100):
        x = X[:, start:start + stepsize, :]
        start += int(0.8*stepsize)
        x_lst.append(x)
        num_step += 1
    x_lst.append(X[:, 900:, :])
    num_step += 1
    num_sample = X.shape[0]
    newX = np.zeros((num_sample, num_step, stepsize, 3))
    for i in range(num_step):
        newX[:,i,:,:] = x_lst[i]
    res = torch.from_numpy(newX).float()
    res = res.transpose(2,3)
    res = res.view(num_sample, num_step, 1, 3, stepsize)
    return res

In [131]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1,128, (2,3), stride = (1,1))
        self.conv2 = nn.Conv2d(128,64, (2,3), stride = (1,2))
        self.conv3 = nn.Conv1d(64,32, 4, stride = 2)
        self.pooling1D = nn.MaxPool1d(3, stride=2)
        self.pooling2D = nn.MaxPool2d((1, 2), stride=(1, 2))
    
    def forward(self, x):
        inputs = self.conv1(x)
        inputs = F.relu(inputs)
        inputs = self.pooling2D(inputs)
        inputs = self.conv2(inputs)
        inputs = F.relu(inputs)
        inputs = inputs.squeeze()
        inputs = self.conv3(inputs)
        inputs = self.pooling1D(inputs)
        inputs = F.relu(inputs)
        
        return inputs

class cnn_lstm(nn.Module):
    def __init__(self, cnn = None, drop_prob = 0.4, n_class = 2, n_layer = 2):
        super(cnn_lstm, self).__init__()
        if(cnn == None):
            self.cnn = CNN()
        else:
            self.cnn = cnn
        self.n_class = n_class
        self.lstm = nn.LSTM(160, 128, num_layers = n_layer, batch_first = True)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, n_class)
        self.dropout = nn.Dropout(drop_prob)
        self.outlayer = nn.Softmax(dim = 2)
        
    def forward(self, x, hidden):
        inputs = transformation(x)
        batch_size, timesteps, C, H, W = inputs.size()
        inputs = inputs.view(batch_size*timesteps,C,H,W)
        c_out = self.cnn(inputs)
        lstm_in = c_out.view(batch_size, timesteps, -1)
        lstm_out, hidden = self.lstm(lstm_in,hidden)
        out = self.dropout(lstm_out)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.fc3(out)
        out = self.outlayer(out)
        
        if(batch_size != 1):
            out = out.transpose(1,2)
        return out, hidden
    
    
    def init_hidden(self,batch_size):
        weight = next(self.parameters()).data
        
        hidden = (weight.new(2, batch_size, 128).zero_(),
                      weight.new(2, batch_size, 128).zero_())
        return hidden

def init_weights(m):
    if type(m) == nn.LSTM:
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'weight_hh' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
    elif type(m) == nn.Conv2d or type(m) == nn.Conv1d or type(m) == nn.Linear:
        torch.nn.init.orthogonal_(m.weight)
        m.bias.data.fill_(0)

In [132]:
def trainBatch(net, trainX, trainY, valX, valY, 
              batch_size = 40, epochs = 200, lr = 0.004):
    opt = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
    #criterion = nn.CrossEntropyLoss(weight= torch.tensor([1,1]))
    criterion = nn.CrossEntropyLoss()
    val_acc = 0
    best_model = None
    for e in range(epochs):
        h = net.init_hidden(batch_size)
        #print(h)
        train_loss = []
        net.train()
        for x,y in take_batch(batch_size, trainX, trainY):
            #print(trainX.shape, trainY.shape)
            targets = torch.from_numpy(y).view(batch_size, -1)
            targets = targets.squeeze()
            #print(targets.size())
            h = tuple([each.data for each in h])
            opt.zero_grad()  
            output, h = net(x, h)
            output = output[:,:,-1]
            #print(output.size())
            loss = criterion(output, targets.long())
            train_loss.append(loss.item())
            loss.backward()
            opt.step()
        acc = check_acc(net, valX, valY)
        if(acc >= val_acc): # store the model performs best on validation set
            best_model = net
        print("Epoch: {}/{}...".format(e+1, epochs),
        "Train Loss: {:.4f}...".format(np.mean(train_loss)),
             "Validation acc: {:.4f}...".format(acc))
    final_model = net
    return best_model, final_model

In [133]:
exps = os.listdir(datapath)

In [134]:
res = {'acc_pure': [], 'acc_100': [], 'acc_200': [], 'acc_300': [], 
       'acc_400': [], 'acc_500': [], 'TPR': [], 'TNR': [], 'best_acc': []}
for exp in exps:
    path = datapath + exp + '/'
    XtrainPure = np.load(file = path + 'train_puredata.npy')
    XtestPure = np.load(file = path + 'test_puredata.npy')
    Xtrain_100 = np.load(file = path + 'Xtrain_100.npy')
    Xtrain_200 = np.load(file = path + 'Xtrain_200.npy')
    Xtrain_300 = np.load(file = path + 'Xtrain_300.npy')
    Xtrain_400 = np.load(file = path + 'Xtrain_400.npy')
    Xtrain_500 = np.load(file = path + 'Xtrain_500.npy')
    Xtest_100 = np.load(file = path + 'Xtest_100.npy')
    Xtest_200 = np.load(file = path + 'Xtest_200.npy')
    Xtest_300 = np.load(file = path + 'Xtest_300.npy')
    Xtest_400 = np.load(file = path + 'Xtest_400.npy')
    Xtest_500 = np.load(file = path + 'Xtest_500.npy')
    
    trainX, trainY, validX, validY = BuildTrainValidset(XtrainPure, Xtrain_100, Xtrain_200, Xtrain_300, Xtrain_400, Xtrain_500)
    
    net = cnn_lstm()
    net.apply(init_weights)
    best, final = trainBatch(net, trainX, trainY, validX, validY)
    
    Ytest_pure = np.zeros((XtestPure.shape[0],1))
    Ytest_syn = np.ones((Xtest_100.shape[0],1))
    TNR = check_acc(final, XtestPure, Ytest_pure)
    
    acc_100 = check_acc(final, Xtest_100, Ytest_syn)
    acc_200 = check_acc(final, Xtest_200, Ytest_syn)
    acc_300 = check_acc(final, Xtest_300, Ytest_syn)
    acc_400 = check_acc(final, Xtest_400, Ytest_syn)
    acc_500 = check_acc(final, Xtest_500, Ytest_syn)
    
    TPR = (acc_100+acc_200+acc_300+acc_400+acc_500)*200/(len(Xtest_100)*5)
    
    res['acc_pure'].append(TNR)
    res['TNR'].append(TNR)
    res['acc_100'].append(acc_100)
    res['acc_200'].append(acc_200)
    res['acc_300'].append(acc_300)
    res['acc_400'].append(acc_400)
    res['acc_500'].append(acc_500)
    res['TPR'].append(TPR)
    
    res['best_acc'].append(check_acc(best, XtestPure, Ytest_pure))
    
    f = open('exp_res.txt', 'w')
    f.write(str(res))
    f.close()

Epoch: 1/200... Train Loss: 0.6925... Validation acc: 0.5450...
Epoch: 2/200... Train Loss: 0.6915... Validation acc: 0.5150...
Epoch: 3/200... Train Loss: 0.6891... Validation acc: 0.5850...
Epoch: 4/200... Train Loss: 0.6862... Validation acc: 0.5500...
Epoch: 5/200... Train Loss: 0.6783... Validation acc: 0.6200...
Epoch: 6/200... Train Loss: 0.6694... Validation acc: 0.6250...
Epoch: 7/200... Train Loss: 0.6547... Validation acc: 0.6750...
Epoch: 8/200... Train Loss: 0.6363... Validation acc: 0.6900...
Epoch: 9/200... Train Loss: 0.6212... Validation acc: 0.7150...
Epoch: 10/200... Train Loss: 0.6152... Validation acc: 0.7250...
Epoch: 11/200... Train Loss: 0.5981... Validation acc: 0.7550...
Epoch: 12/200... Train Loss: 0.5928... Validation acc: 0.7450...
Epoch: 13/200... Train Loss: 0.5909... Validation acc: 0.7600...
Epoch: 14/200... Train Loss: 0.5858... Validation acc: 0.7700...
Epoch: 15/200... Train Loss: 0.5861... Validation acc: 0.7550...
Epoch: 16/200... Train Loss: 0.584

Epoch: 127/200... Train Loss: 0.4809... Validation acc: 0.7100...
Epoch: 128/200... Train Loss: 0.4748... Validation acc: 0.7350...
Epoch: 129/200... Train Loss: 0.4711... Validation acc: 0.6650...
Epoch: 130/200... Train Loss: 0.4614... Validation acc: 0.7500...
Epoch: 131/200... Train Loss: 0.4541... Validation acc: 0.7050...
Epoch: 132/200... Train Loss: 0.4673... Validation acc: 0.6800...
Epoch: 133/200... Train Loss: 0.4661... Validation acc: 0.7100...
Epoch: 134/200... Train Loss: 0.4443... Validation acc: 0.6700...
Epoch: 135/200... Train Loss: 0.4445... Validation acc: 0.7350...
Epoch: 136/200... Train Loss: 0.4458... Validation acc: 0.6850...
Epoch: 137/200... Train Loss: 0.4363... Validation acc: 0.7150...
Epoch: 138/200... Train Loss: 0.4471... Validation acc: 0.7000...
Epoch: 139/200... Train Loss: 0.4399... Validation acc: 0.6900...
Epoch: 140/200... Train Loss: 0.4273... Validation acc: 0.6900...
Epoch: 141/200... Train Loss: 0.4209... Validation acc: 0.7000...
Epoch: 142

Epoch: 53/200... Train Loss: 0.5493... Validation acc: 0.7700...
Epoch: 54/200... Train Loss: 0.5492... Validation acc: 0.7550...
Epoch: 55/200... Train Loss: 0.5479... Validation acc: 0.7700...
Epoch: 56/200... Train Loss: 0.5518... Validation acc: 0.7650...
Epoch: 57/200... Train Loss: 0.5532... Validation acc: 0.7700...
Epoch: 58/200... Train Loss: 0.5492... Validation acc: 0.7700...
Epoch: 59/200... Train Loss: 0.5455... Validation acc: 0.7750...
Epoch: 60/200... Train Loss: 0.5485... Validation acc: 0.7600...
Epoch: 61/200... Train Loss: 0.5447... Validation acc: 0.7600...
Epoch: 62/200... Train Loss: 0.5428... Validation acc: 0.7700...
Epoch: 63/200... Train Loss: 0.5485... Validation acc: 0.7700...
Epoch: 64/200... Train Loss: 0.5468... Validation acc: 0.7750...
Epoch: 65/200... Train Loss: 0.5476... Validation acc: 0.7700...
Epoch: 66/200... Train Loss: 0.5518... Validation acc: 0.7700...
Epoch: 67/200... Train Loss: 0.5504... Validation acc: 0.7650...
Epoch: 68/200... Train Lo

Epoch: 178/200... Train Loss: 0.4687... Validation acc: 0.6800...
Epoch: 179/200... Train Loss: 0.4965... Validation acc: 0.7050...
Epoch: 180/200... Train Loss: 0.4704... Validation acc: 0.7300...
Epoch: 181/200... Train Loss: 0.4604... Validation acc: 0.7300...
Epoch: 182/200... Train Loss: 0.4492... Validation acc: 0.7400...
Epoch: 183/200... Train Loss: 0.4424... Validation acc: 0.7550...
Epoch: 184/200... Train Loss: 0.4451... Validation acc: 0.7350...
Epoch: 185/200... Train Loss: 0.4538... Validation acc: 0.7350...
Epoch: 186/200... Train Loss: 0.4421... Validation acc: 0.7550...
Epoch: 187/200... Train Loss: 0.4423... Validation acc: 0.7200...
Epoch: 188/200... Train Loss: 0.4649... Validation acc: 0.7250...
Epoch: 189/200... Train Loss: 0.4575... Validation acc: 0.7400...
Epoch: 190/200... Train Loss: 0.4665... Validation acc: 0.6700...
Epoch: 191/200... Train Loss: 0.4765... Validation acc: 0.7100...
Epoch: 192/200... Train Loss: 0.4700... Validation acc: 0.7550...
Epoch: 193

Epoch: 104/200... Train Loss: 0.5113... Validation acc: 0.7900...
Epoch: 105/200... Train Loss: 0.5239... Validation acc: 0.7600...
Epoch: 106/200... Train Loss: 0.5186... Validation acc: 0.7800...
Epoch: 107/200... Train Loss: 0.5045... Validation acc: 0.7900...
Epoch: 108/200... Train Loss: 0.5133... Validation acc: 0.7750...
Epoch: 109/200... Train Loss: 0.5023... Validation acc: 0.7900...
Epoch: 110/200... Train Loss: 0.5036... Validation acc: 0.7850...
Epoch: 111/200... Train Loss: 0.5070... Validation acc: 0.7750...
Epoch: 112/200... Train Loss: 0.5019... Validation acc: 0.7850...
Epoch: 113/200... Train Loss: 0.4977... Validation acc: 0.7850...
Epoch: 114/200... Train Loss: 0.4928... Validation acc: 0.7750...
Epoch: 115/200... Train Loss: 0.4989... Validation acc: 0.7800...
Epoch: 116/200... Train Loss: 0.4960... Validation acc: 0.7800...
Epoch: 117/200... Train Loss: 0.4984... Validation acc: 0.8000...
Epoch: 118/200... Train Loss: 0.4937... Validation acc: 0.7800...
Epoch: 119

...

Epoch: 183/200... Train Loss: 0.5485... Validation acc: 0.6200...
Epoch: 184/200... Train Loss: 0.4945... Validation acc: 0.8150...
Epoch: 185/200... Train Loss: 0.4385... Validation acc: 0.8150...
Epoch: 186/200... Train Loss: 0.4360... Validation acc: 0.8250...
Epoch: 187/200... Train Loss: 0.4333... Validation acc: 0.8100...
Epoch: 188/200... Train Loss: 0.4357... Validation acc: 0.8000...
Epoch: 189/200... Train Loss: 0.4382... Validation acc: 0.7850...
Epoch: 190/200... Train Loss: 0.4270... Validation acc: 0.8050...
Epoch: 191/200... Train Loss: 0.4331... Validation acc: 0.8000...
Epoch: 192/200... Train Loss: 0.4322... Validation acc: 0.7900...
Epoch: 193/200... Train Loss: 0.4372... Validation acc: 0.8100...
Epoch: 194/200... Train Loss: 0.4353... Validation acc: 0.8200...
Epoch: 195/200... Train Loss: 0.4319... Validation acc: 0.8100...
Epoch: 196/200... Train Loss: 0.4336... Validation acc: 0.8100...
Epoch: 197/200... Train Loss: 0.4318... Validation acc: 0.8100...
Epoch: 198

Epoch: 109/200... Train Loss: 0.5436... Validation acc: 0.8000...
Epoch: 110/200... Train Loss: 0.5375... Validation acc: 0.7850...
Epoch: 111/200... Train Loss: 0.5329... Validation acc: 0.8050...
Epoch: 112/200... Train Loss: 0.5353... Validation acc: 0.7850...
Epoch: 113/200... Train Loss: 0.5257... Validation acc: 0.8050...
Epoch: 114/200... Train Loss: 0.5303... Validation acc: 0.7850...
Epoch: 115/200... Train Loss: 0.5250... Validation acc: 0.7900...
Epoch: 116/200... Train Loss: 0.5287... Validation acc: 0.7650...
Epoch: 117/200... Train Loss: 0.5382... Validation acc: 0.8000...
Epoch: 118/200... Train Loss: 0.5241... Validation acc: 0.7950...
Epoch: 119/200... Train Loss: 0.5200... Validation acc: 0.8000...
Epoch: 120/200... Train Loss: 0.5278... Validation acc: 0.7850...
Epoch: 121/200... Train Loss: 0.5253... Validation acc: 0.8100...
Epoch: 122/200... Train Loss: 0.5203... Validation acc: 0.8050...
Epoch: 123/200... Train Loss: 0.5326... Validation acc: 0.8050...
Epoch: 124

Epoch: 34/200... Train Loss: 0.5739... Validation acc: 0.7350...
Epoch: 35/200... Train Loss: 0.5676... Validation acc: 0.7400...
Epoch: 36/200... Train Loss: 0.5695... Validation acc: 0.7100...
Epoch: 37/200... Train Loss: 0.5626... Validation acc: 0.7050...
Epoch: 38/200... Train Loss: 0.5651... Validation acc: 0.7200...
Epoch: 39/200... Train Loss: 0.5610... Validation acc: 0.7350...
Epoch: 40/200... Train Loss: 0.5612... Validation acc: 0.7300...
Epoch: 41/200... Train Loss: 0.5638... Validation acc: 0.7300...
Epoch: 42/200... Train Loss: 0.5600... Validation acc: 0.7350...
Epoch: 43/200... Train Loss: 0.5671... Validation acc: 0.7300...
Epoch: 44/200... Train Loss: 0.5622... Validation acc: 0.7400...
Epoch: 45/200... Train Loss: 0.5578... Validation acc: 0.7400...
Epoch: 46/200... Train Loss: 0.5631... Validation acc: 0.7350...
Epoch: 47/200... Train Loss: 0.5613... Validation acc: 0.7150...
Epoch: 48/200... Train Loss: 0.5608... Validation acc: 0.7350...
Epoch: 49/200... Train Lo

Epoch: 160/200... Train Loss: 0.4147... Validation acc: 0.7700...
Epoch: 161/200... Train Loss: 0.4225... Validation acc: 0.7150...
Epoch: 162/200... Train Loss: 0.4199... Validation acc: 0.7600...
Epoch: 163/200... Train Loss: 0.4125... Validation acc: 0.7800...
Epoch: 164/200... Train Loss: 0.4283... Validation acc: 0.7450...
Epoch: 165/200... Train Loss: 0.4185... Validation acc: 0.7450...
Epoch: 166/200... Train Loss: 0.4216... Validation acc: 0.7300...
Epoch: 167/200... Train Loss: 0.4069... Validation acc: 0.7550...
Epoch: 168/200... Train Loss: 0.4085... Validation acc: 0.7650...
Epoch: 169/200... Train Loss: 0.4334... Validation acc: 0.7250...
Epoch: 170/200... Train Loss: 0.4104... Validation acc: 0.7500...
Epoch: 171/200... Train Loss: 0.4095... Validation acc: 0.7800...
Epoch: 172/200... Train Loss: 0.4158... Validation acc: 0.7250...
Epoch: 173/200... Train Loss: 0.4137... Validation acc: 0.7650...
Epoch: 174/200... Train Loss: 0.4087... Validation acc: 0.7650...
Epoch: 175

Epoch: 86/200... Train Loss: 0.5243... Validation acc: 0.7750...
Epoch: 87/200... Train Loss: 0.5290... Validation acc: 0.7500...
Epoch: 88/200... Train Loss: 0.5234... Validation acc: 0.8000...
Epoch: 89/200... Train Loss: 0.5277... Validation acc: 0.7650...
Epoch: 90/200... Train Loss: 0.5253... Validation acc: 0.7500...
Epoch: 91/200... Train Loss: 0.5271... Validation acc: 0.7850...
Epoch: 92/200... Train Loss: 0.5209... Validation acc: 0.7650...
Epoch: 93/200... Train Loss: 0.5175... Validation acc: 0.7700...
Epoch: 94/200... Train Loss: 0.5226... Validation acc: 0.7100...
Epoch: 95/200... Train Loss: 0.5257... Validation acc: 0.7700...
Epoch: 96/200... Train Loss: 0.5197... Validation acc: 0.7700...
Epoch: 97/200... Train Loss: 0.5117... Validation acc: 0.7550...
Epoch: 98/200... Train Loss: 0.5008... Validation acc: 0.7650...
Epoch: 99/200... Train Loss: 0.5001... Validation acc: 0.7850...
Epoch: 100/200... Train Loss: 0.4913... Validation acc: 0.7550...
Epoch: 101/200... Train 

In [135]:
res['TNR']

[0.62, 0.83, 0.79, 0.74, 0.765, 0.675, 0.845, 0.915, 0.845, 0.695]

In [136]:
res['TPR']

[0.727,
 0.614,
 0.622,
 0.673,
 0.634,
 0.719,
 0.621,
 0.6250000000000001,
 0.619,
 0.752]

In [137]:
np.mean(res['TNR'])

0.772

In [138]:
np.mean(res['TPR'])

0.6606